
<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Three: Classifier training and performance assessment. </h2>	

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [6]:
def train_test_split(df):
    return train_test_split(df, test_size=0.2, random_state=42)
#Split the data into a training set, and test set 
def accuracy(pred, actual):
    return np.sum(np.equal(pred, actual))/len(pred)
# Calculate the accuracy percentage of the predicted values

<h3>  Train models using all of the following methods below. Be sure to drop the actual image column, and the encoding</h3>	Take note of the differences in accuracy, and methods.


Logistic Regression

In [8]:
data = read_organize_data("20_categories_training/20_categories_training")
data = feature_frame(data)
X_train, Y_train = train_test_split(data)

NameError: name 'read_organize_data' is not defined

In [3]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(multi_class='ovr', penalty='l1', C=1.0).fit(X_train, Y_train)
training_accuracy = accuracy(log_model.predict(X_train), Y_train)
print(training_accuracy)

NameError: name 'X_train' is not defined

K-nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
n_neighbors = 5
knn_model = KNeighborsClassifier(n_neighbors=n_neighbors).fit(X_train, Y_train)
training_accuracy = accuracy(knn_model.predict(X_train), Y_train)
print(training_accuracy)

Classification Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree_model = DecisionTreeClassifier(max_depth=None).fit(X_train, Y_train)
training_accuracy = accuracy(dtree_model.predict(X_train), Y_train)
print(training_accuracy)

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier(n_estimators=100, 
                                      bootstrap = True, 
                                      max_features = 'sqrt', 
                                      max_depth=None).fit(X_train, Y_train)
training_accuracy = accuracy(forest_model.predict(X_train), Y_train)
print(training_accuracy)

Support Vector Machine

In [ ]:
from sklearn.svm import SVC
svc_model = SVC(C=1.0, tol=1e-3).fit(X_train, Y_train)
training_accuracy = accuracy(svc_model.predict(X_train), Y_train)
print(training_accuracy)